In [23]:
import os
from json import JSONDecodeError
from openai import OpenAI
import json
import csv
from tqdm import tqdm
import re
from collections import Counter
from load_frequencies_for_study import get_word_frequency_percentiles

In [24]:
client = OpenAI(api_key="")

rare_word_percentage = 0.05 # Follow the 95%/98% rule, but consider 3% and 4% as well
SOURCE_LANGUAGE = 'English'
TARGET_LANGUAGE = 'Spanish'
RARE_WORD_FREQUENCY = 2

folder_path = './articles'

articles_dict = dict()

for file_name in os.listdir(folder_path):

    if file_name.endswith('.txt'):
        file_path = os.path.join(folder_path, file_name)

        with open(file_path, 'r', encoding='utf-8') as file:
            content = file.read()

        article_name = os.path.splitext(file_name)[0]
        articles_dict[article_name] = dict()

        articles_dict[article_name]['article_text'] = content
        articles_dict[article_name]['article_length'] = len(content.split())
        articles_dict[article_name]['number_of_rare_words'] = int(len(content.split()) * rare_word_percentage) * 2

In [25]:
articles_dict.keys()

dict_keys(['article_1', 'article_2', 'article_3'])

In [26]:
def parse_rare_words(article_data):
    try:
        parsed_data = json.loads(article_data)
    except json.JSONDecodeError:
        parsed_data = manually_parse_rare_words(article_data)

    return parsed_data


def manually_parse_rare_words(article_data):
    parsed_data = {}

    for line in article_data.splitlines():
        if ':' in line:
            key, value = line.split(':', 1)
            parsed_data[key.strip()] = value.strip()

    return parsed_data

In [27]:
def generate_rare_words(article_dict):

    # TODO: words should not appear in day to day speech
    # TODO: should be able to insert into the text
    # TODO: use markdown

    word_gen_system_prompt = f"""You are a bilingual language expert. Your task is to analyze the provided article and suggest rare nouns, which are rare so they don't appear in day-to-day speech and could be inserted/used as part of that article. The words you suggest would likely be unfamiliar to intermediate language learners and potentially unknown to native speakers.

# Key Requirements
1. Analyze the provided article and generate the list of {article_dict["number_of_rare_words"]} {TARGET_LANGUAGE} rare nouns.
2. Ensure these rare words are nouns between 2-5 syllables, not overly technical, and naturally fit the article's context.
3. Ensure that the rare nouns in the {TARGET_LANGUAGE} and their translations in {SOURCE_LANGUAGE} are significantly different in form and not too similar to each other.

Provide your output in valid JSON."""

    word_gen_user_prompt = f"""Analyze the following article and identify the {article_dict['number_of_rare_words']} rare nouns.

For each of the {article_dict['number_of_rare_words']} rare nouns, provide:
The rare word in {TARGET_LANGUAGE}
Its translation in {SOURCE_LANGUAGE}, if a word has multiple translations provide the one that is most similar in form.

Format the output as a JSON array with the following structure:
{{
    "rare_words": [
        {{
            "rare_word": "<word in {TARGET_LANGUAGE}>",
            "translation": "<the translation of the word into {SOURCE_LANGUAGE}>",
            "examples": ["example of where could be inserted in the text", "another example"]
        }}
    ]
}}

Article:
{article_dict['article_text']}

REMEMBER: YOU MUST ENSURE THAT THE FORM OF THE RARE WORDS IN {TARGET_LANGUAGE} AND THEIR TRANSLATIONS IN {SOURCE_LANGUAGE} ARE SIGNIFICANTLY DIFFERENT SO USERS CAN'T GUESS THE TRANSLATION."""

    response = client.chat.completions.create(
        model="gpt-4o",
        response_format={"type": "json_object"},
        messages=[
            {"role": "system", "content": word_gen_system_prompt.strip()},
            {"role": "user", "content": word_gen_user_prompt.strip()}
        ]
    )

    try:
        final_response = parse_rare_words(response.choices[0].message.content)
    except JSONDecodeError:
        final_response = manually_parse_rare_words(response.choices[0].message.content)

    return final_response['rare_words']


# TODO: Control forms, Controls length, Rare, Having different sources - no leg up, Not latin-ate in rooting, germanic
# TODO: Spanish L2, English L1
# word_list_text = generate_rare_words()
# print("Generated Rare Nouns:\n")
# print(word_list_text)
# print("\n" + "-"*80 + "\n")

In [28]:
def repurpose_article(articles_dict, rare_words):
    system_prompt = f"""You are language expert. Your task is to:

# Guidelines:
1. Rewrite the article using a very simple {TARGET_LANGUAGE}, every word in the text should be understandable by a beginner language learner.
2. Incorporate the given rare nouns into the simplified text according to the specified frequency, ensuring each word appears the same number of times and is spread out throughout the text. You must ensure that each rare word is used {RARE_WORD_FREQUENCY} times in the simplified text.
3. Bold all inserted rare words using **word** format

# Parameters:
- Word Frequency: How often to insert rare words
- Article: The original article to simplify
- Rare Words List: The list of rare words to incorporate into the simplified text"""

    user_prompt = f"""Please follow these steps:
1. Rephrase this article using very simple {TARGET_LANGUAGE}, the entire article should be understandable by a beginner language learner, avoid any even slightly uncommon words. You can add paragraphs if needed.
2. Insert the provided rare nouns according to the specified frequency. Each rare word MUST appear {RARE_WORD_FREQUENCY} times in the simplified text.

# Parameters:
- Rare Word Frequency: {RARE_WORD_FREQUENCY} times for each word
- Rare Words List: {rare_words}
- Article: {articles_dict['article_text']}

REMEMBER: YOU MUST ENSURE THAT EACH RARE WORD IN THE RARE WORD LIST {rare_words} IS USED {RARE_WORD_FREQUENCY} TIMES IN THE SIMPLIFIED TEXT. NO MORE, NO LESS.

Bold all inserted rare words using **rare word** format.

Finally print out how many times each word is used in the text. Remember each word should appear {RARE_WORD_FREQUENCY} times in the text.

Do not modify the number"""

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": system_prompt.strip()},
            {"role": "user", "content": user_prompt.strip()}
        ]
    )

    return response.choices[0].message.content.strip()

In [29]:
import Levenshtein

def similarity_ratio(word1, word2):
    return Levenshtein.ratio(word1, word2)


def filter_similar_word_translations(rare_words, threshold=0.8):
    filtered = []
    for entry in rare_words:
        word = entry['rare_word'].lower()
        translation = entry['translation'].lower()
        if not word or not translation:
            continue  # Skip if either is missing
        
        similarity = similarity_ratio(word, translation)
        if similarity < threshold:
            filtered.append(entry)
        # else:
        #     print(f"Removing '{word}' and its translation '{translation}' due to high similarity ({similarity:.2f})")
    return filtered

In [30]:
def main():
    for article_name, article_dict in tqdm([list(articles_dict.items())[2]]):

        rare_words = generate_rare_words(article_dict)

        # Step 2: Get frequency percentiles for translations
        translations = [rare_word['rare_word'] for rare_word in rare_words]
        rare_word_ranks = get_word_frequency_percentiles(translations, TARGET_LANGUAGE)

        # Step 3: Filter by frequency percentile (>15)
        filtered_by_frequency = [
            word for word in rare_words 
            if rare_word_ranks.get(word['rare_word'], 0) > 35 and ' ' not in word['rare_word']
        ]

        final_filtered_rare_words = []
        while len(final_filtered_rare_words) < 8:
            final_filtered_rare_words = filter_similar_word_translations(filtered_by_frequency, threshold=0.55)

        # Optional: Print the final list
        print(final_filtered_rare_words)

        final_filtered_rare_words_list = [entry['rare_word'] for entry in final_filtered_rare_words]

        article_text_with_rare_words = repurpose_article(articles_dict[article_name], " ".join(final_filtered_rare_words_list))
        # article_text_with_rare_words = article_text_with_rare_words.replace('**', '') # Remove the bold formatting for now

        articles_dict[article_name]['rare_words'] = [rare_word for rare_word in rare_words if rare_word['rare_word'] in final_filtered_rare_words_list]
        articles_dict[article_name]['article_text_with_rare_words'] = article_text_with_rare_words

        # Save the rare words to a CSV file
        with open(f'./rare_words/{article_name}_rare_words.csv', 'w', newline='', encoding='utf-8') as csv_file:
            writer = csv.writer(csv_file)

            writer.writerow(['rare_word', 'translation'])
            for item in articles_dict[article_name]['rare_words']:
                writer.writerow([item['rare_word'], item['translation']])

        # Save the text with rare words to a text file
        with open(f'./articles_with_rare_words/{article_name}_with_rare_words.txt', 'w', encoding='utf-8') as f:
            f.write(article_text_with_rare_words)

        return articles_dict[article_name]

    with open('./articles_with_rare_words.json', 'w', encoding='utf-8') as json_file: # TODO: should we include datetime in the file name?
        json.dump(articles_dict, json_file, ensure_ascii=False, indent=4)


if __name__ == '__main__':
    article = main()
    text = article['article_text_with_rare_words']
    rare_words_dict = article['rare_words']
    
    print("Rare word dict")
    print(rare_words_dict)
    print("####")

    translation_to_original = {
        entry['rare_word'].lower(): entry['translation'] for entry in rare_words_dict
    }

    rare_words = set(translation_to_original.keys())
    cleaned_text = re.sub(r'[^\w\s]', '', text)  
    filtered_words = cleaned_text.lower().split()

    rare_word_ranks_for_final_text = get_word_frequency_percentiles(filtered_words, TARGET_LANGUAGE)
    words_that_are_too_rare = set()

    for word in filtered_words:
        if rare_word_ranks_for_final_text.get(word, 0) > 15 and word.lower() not in rare_words and word.lower() not in ['didnt', 'isnt'] and not word.isdigit():
            words_that_are_too_rare.add((word, f"Rank: {rare_word_ranks_for_final_text[word]}"))

    print("Words in simplified text that are too rare:")
    print(words_that_are_too_rare)

    word_counts = Counter(filtered_words)
    rare_word_counts = {translation: 0 for translation in rare_words}

    for translation in rare_word_counts:
        rare_word_counts[translation] = word_counts.get(translation, 0) - 1

    rare_word_counts = {k: v for k, v in rare_word_counts.items() if v > 0}
    total_rare_word_count = sum(rare_word_counts.values())

    print()
    print(text)

    print("\nRare Words and Their Counts:")
    for translation, count in rare_word_counts.items():
        original = translation_to_original.get(translation, "Unknown Original")
        print(f"{translation} ({original}): {count}")

    print(f"\nTotal Rare Word Count: {total_rare_word_count}")
    print(f"Article Length: {len(text.split())} words")
    if len(text.split()) > 0:
        percentage = (total_rare_word_count / len(text.split())) * 100
        print(f"Percentage of Rare Words: {percentage:.2f}%")
    else:
        print("Article is empty. Cannot compute percentage of rare words.")

  0%|          | 0/1 [00:06<?, ?it/s]


KeyboardInterrupt: 

In [31]:
from collections import defaultdict

text = """Carlos es músico y toca la guitarra. Ha viajado mucho con su música, creando melodías tan coloridas como un **arcoíris**. Toca la guitarra con mucha emoción en España y América Latina. Cuando se presenta, usa un **atavío** elegante. Carlos muestra su pasión por la música tocando en el **bordillo** de las calles. Tiene más de veinte discos y es muy famoso en la música española. Carlos nació en Valencia en 1975. Él era el hijo más joven de cinco hermanos. Desde niño, le gustaba mucho la música. Aprendió a tocar la guitarra con su abuelo que era músico de flamenco. A los ocho años, ya tocaba canciones difíciles de flamenco. Pasaba mucho tiempo escuchando música en la radio.

Su técnica única combinaba la fuerza del **bordón**, la cuerda más baja de la guitarra, con una gran precisión en las notas altas, como un **velamen** que atrapa cada brisa.Esta forma especial de tocar la tenía en la sangre, tras largas noches practicando con su abuelo en la terraza de su casa valenciana, donde el sonido de su guitarra se mezclaba con el aroma de los árboles de naranja y el suave sonido del mar Mediterráneo. Los críticos musicales se sorprendían por su capacidad para hacer que la guitarra "cantara" con una voz propia, una habilidad que pocos guitarristas habían logrado dominar, convirtiéndose así en un **baluarte** de la música moderna.

En 1995, Carlos lanzó su primer álbum llamado Alma y Raíces. La gente lo recibió con entusiasmo porque flamenco y jazz, lo cual fue como un suave **velamen** que los llevaba a nuevas experiencias musicales. Con sonidos auténticos y originales, Carlos capturó la esencia de sus raíces en cada canción, lo que le ayudó a trabajar con otros artistas famosos. Pronto comenzó una gira por España, y en 1998, llevó su recorrido a México y Argentina, donde su música fue muy apreciada por el público de esos países.

Carlos también trabajó en el cine. Hizo música para películas y actuó en papeles pequeños. Su primera película fue Corazón en 2001, donde era un músico. La película tuvo éxito y, como un **arcoíris** después de la tormenta, su talento se puso aún más de manifiesto cuando ganó un premio Goya en 2005 por la mejor música de película.

Carlos es una persona humilde que ama sus raíces. En Sevilla en 2002, conoció a su esposa Laura, una bailarina de flamenco. Se casaron en 2003 en una ceremonia pequeña en Andalucía. Los dos han trabajado juntos en música y teatro, donde Carlos toca el **bordón** de su guitarra con habilidad, añadiendo un toque profundo a sus interpretaciones, que son muy queridas en España.

Carlos ha ganado muchos premios en su carrera. En 2012, recibió la Orden de las Artes del Ministerio de Cultura español. Le gusta llevar el flamenco a nuevas personas. Su último disco, Raíces Nuevas, salió en 2020. Este disco tiene ritmos latinos, flamenco y música electrónica. Con cada golpe en el **timbal**, Carlos siente una conexión profunda con sus raíces, como si el sonido se sintiera en cada rincón de su alma. Esta unión de ritmos le trae **júbilo** y le recuerda por qué ha dedicado su vida a la música. Para él, el flamenco no solo es un estilo, sino un **baluarte** de identidad, algo que muestra a sus jóvenes estudiantes, enseñándoles la importancia de cuidar el legado cultural a través de la pasión y el arte. Le gusta caminar en la sierra valenciana, porque siempre encuentra paz y **júbilo** en la naturaleza. También le gusta el arte y pasar tiempo con amigos y familia. Aunque todavía hace algunos conciertos, ha reducido su carga de trabajo.

Con treinta años en la música, Carlos es un ejemplo de amor a su arte. Su pasión por el flamenco lo hace un músico especial. Aunque trabaja menos, su música y legado siguen vivos, enseñando a jóvenes artistas. En sus actuaciones viste un elegante **atavío** teatral que mejora sus presentaciones.

Carlos es un símbolo de la evolución de la música española. A través de su arte, ha compartido la esencia del flamenco y la ha mezclado con otros estilos de manera única. Aunque se ha alejado del **bordillo** de las calles, sigue motivando a jóvenes músicos, enseñándoles la importancia de sus raíces y de explorar nuevas formas de expresión. Su legado vive en su música y en quienes aprenden de él. Cada día, más y más gente baila y canta sus canciones con mucha alegría."""



def find_and_count_encircled_words(text):
    # Regular expression to find words between double asterisks
    pattern = r"\*\*(\w+)\*\*"
    words = re.findall(pattern, text)
    
    # Count occurrences of each word
    word_counts = Counter(words)
    
    return word_counts

# Sample text
print(len(text.split()))

# Find and print encircled words
encircled_words = find_and_count_encircled_words(text)
print("Encircled words:", encircled_words)
print(len(encircled_words))

742
Encircled words: Counter({'arcoíris': 2, 'atavío': 2, 'bordillo': 2, 'bordón': 2, 'velamen': 2, 'baluarte': 2, 'júbilo': 2, 'timbal': 1})
8


In [32]:
def find_base_word(rare_word):
    system_prompt = f"""You are an Spanish language expert. For this {rare_word} give me its base form. So if it's a noun, give me the singular form. If it's a verb, give me the infinitive form.

Just the base do not reply with anything else"""

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "user", "content": system_prompt.strip()},
        ]
    )

    return response.choices[0].message.content.strip().lower()

In [33]:
find_base_word("Presagiaba")

'presagiar'

In [38]:
text = """Ana Belén, una cantante, directora y actriz famosa en España, ha lanzado más de treinta y cinco discos, actuado en treinta películas y veinte obras de teatro. Ha dado conciertos por toda España y Latinoamérica, dejando un rastro **tornasol** de emociones en cada presentación. Ana ha defendido causas sociales a través de su arte, interpretando canciones sobre paz, derechos humanos y justicia social, despertando un **embeleso** en su público que va más allá de la música. Esto la ha convertido en una figura respetada, tanto en el mundo artístico como en los movimientos de cambio.

A lo largo de su carrera, Ana Belén ha recibido muchos reconocimientos por su talento en las artes. Sin embargo, ella nunca ha considerado estos honores como una simple **lisonja**; para Ana, cada premio representa un reconocimiento al esfuerzo y al **desvelo** constante en su carrera. Su enfoque ha sido siempre seguir aprendiendo y mejorando, sin dejarse llevar por los aplausos ni quedarse con los éxitos obtenidos. Este compromiso con su arte y su público ha sido una de las cosas que la han mantenido relevante y respetada en el mundo cultural.

Ana Belén encontró en el cine un **vericueto** inesperado hacia el universo de las artes. Aunque había comenzado en la música, su experiencia en el cine infantil la llevó a descubrir nuevas partes de sí misma y de su talento. Fue en este recorrido donde desarrolló una capacidad única para tomar diferentes personajes y conectarse emocionalmente con el público. Este camino no solo fue un camino profesional, sino también una oportunidad para explorar sus habilidades como intérprete, algo que marcaría profundamente su carrera.

A los doce años, empezó a estudiar música en una academia conocida de Madrid. A los trece, participó en una película musical para niños que tuvo mucho éxito. El cine fue su **vericueto** hacia el teatro. Su éxito en el cine juvenil le dio confianza para intentar nuevos proyectos. Aunque era joven, Ana demostró gran profesionalismo y un talento natural. Su habilidad para conectar con el público desde muy joven prometía una carrera en las artes.

En 1971 conoció al cantante Víctor Manuel durante una gira de teatro en Galicia. Pronto actuaron juntos en una película. En 1972, Ana y Víctor se casaron en Gibraltar. Después de casarse, Ana volvió a cantar y lanzó su primer disco con canciones de Víctor. La **alquimia** de su música encantó a muchos.

Ana Belén ha trabajado mucho en cine, y con gran **entereza** ha sido también directora de teatro y actriz de televisión. Su trabajo es tan bueno que Francia le dio un premio importante (la Orden de las Artes) en 1986 y España le dio otro (la Medalla de Oro) en 2007. La **alquimia** que crea entre estas tres formas de actuar la hace muy especial.

Ahora, Ana no trabaja tanto y puede disfrutar más. Le encanta ir al gimnasio temprano y también disfruta ir al cine. Además, le gusta ver exposiciones de pintura que muestran **tornasol** de colores. Ana pasas mucho su tiempo con su familia y amigos, y después de tantos años de **desvelo** y esfuerzo, finalmente siente que puede usar su tiempo a lo que realmente ama. Su vida es más tranquila ahora. Cada día aprecia más los pequeños momentos que la vida le regala y sonríe al recordar el camino recorrido.

A lo largo de su carrera, Ana enfrentó muchos retos, pero siempre mostró gran **entereza**. Su carrera fue como una **lisonja** de logros. Para Ana, cada desafío ha sido una oportunidad para crecer. Considera que la fortaleza que mostró durante los momentos difíciles fue clave en su evolución como artista y persona. Su historia es una fuente de inspiración para todos aquellos que persiguen sus sueños a pesar de los obstáculos.

Durante su carrera, Ana Belén ha sido una **incógnita** fascinante para sus seguidores y críticos. Siempre buscó nuevas ideas para crear y exploró diferentes artes. Nunca se quedó en un solo estilo, y esa evolución constante la ha mantenido relevante con el tiempo. Su vida estuvo llena de **embeleso** por el arte. Para muchos, cada nuevo proyecto de Ana es una sorpresa, un misterio por descubrir que la ha hecho una de las personas más interesantes de la cultura española. Ella sigue siendo una **incógnita** en el mundo del arte. Ana Belén es una artista única, llena de pasión y compromiso. Su camino muestra que con esfuerzo y amor por el arte se puede lograr mucho."""


def find_and_count_encircled_words(text):
    # Regular expression to find words between double asterisks
    pattern = r"\*\*(\w+)\*\*"
    words = re.findall(pattern, text)
    
    # Count occurrences of each word
    word_counts = Counter(words)
    
    return word_counts


# Sample text
# Clean the text and filter words
cleaned_text = re.sub(r'[^\w\s]', '', text)  
filtered_words = cleaned_text.lower().split()

# Get the encircled words and treat them as rare words
encircled_words = find_and_count_encircled_words(text)
rare_words = set(encircled_words.keys())

# Get word frequency ranks for filtered words
TARGET_LANGUAGE = "Spanish"  # replace with actual target language
rare_word_ranks_for_final_text = get_word_frequency_percentiles(filtered_words, TARGET_LANGUAGE)
words_that_are_too_rare = set()

# Identify words that are too rare
for word in set(filtered_words):
    if rare_word_ranks_for_final_text.get(word, 0) > 20 and word.lower() not in rare_words and word.lower() not in ['didnt', 'isnt'] and not word.isdigit():
        print("-----------------------------")
        print(f"Encircled word '{word}' is treated as rare")
        base_word = find_base_word(word)
        base_word = re.sub(r'[^\w\s]', '', base_word)
        print(f"Base word for '{word}': {base_word}")
        if get_word_frequency_percentiles([base_word], TARGET_LANGUAGE)[base_word] > 20:
            print("################")
            print("Still a rare word")
            print(f"Base word for '{word}': {base_word}")
            words_that_are_too_rare.add((word, f"Rank: {rare_word_ranks_for_final_text[word]}"))

        print("-----------------------------")
# Print results
print("Total words in text:", len(text.split()))
print("Rare words:", encircled_words)
print("Words that are too rare:", words_that_are_too_rare)
print("Number of too-rare words:", len(words_that_are_too_rare))

-----------------------------
Encircled word 'enfrentó' is treated as rare
Base word for 'enfrentó': enfrentar
-----------------------------
-----------------------------
Encircled word 'belén' is treated as rare
Base word for 'belén': belén
################
Still a rare word
Base word for 'belén': belén
-----------------------------
-----------------------------
Encircled word 'conectarse' is treated as rare
Base word for 'conectarse': conectar
-----------------------------
-----------------------------
Encircled word 'prometía' is treated as rare
Base word for 'prometía': prometer
-----------------------------
-----------------------------
Encircled word 'persiguen' is treated as rare
Base word for 'persiguen': perseguir
-----------------------------
-----------------------------
Encircled word 'galicia' is treated as rare
Base word for 'galicia': galicia
################
Still a rare word
Base word for 'galicia': galicia
-----------------------------
-----------------------------
En

In [228]:
rare_word_ranks_for_final_text.get("poseer", 0)

0

In [22]:
text = """Ana Belén, una cantante, directora y actriz famosa en España, ha lanzado más de treinta y cinco discos, actuado en treinta películas y veinte obras de teatro. Ha dado conciertos por toda España y Latinoamérica, dejando un rastro **tornasol** de emociones en cada presentación. Ana ha defendido causas sociales a través de su arte, interpretando canciones sobre paz, derechos humanos y justicia social, despertando un **embeleso** en su público que va más allá de la música. Esto la ha convertido en una figura respetada, tanto en el mundo artístico como en los movimientos de cambio.

A lo largo de su carrera, Ana Belén ha recibido muchos reconocimientos por su talento en las artes. Sin embargo, ella nunca ha considerado estos honores como una simple **lisonja**; para Ana, cada premio representa un reconocimiento al esfuerzo y al **desvelo** constante en su carrera. Su enfoque ha sido siempre seguir aprendiendo y mejorando, sin dejarse llevar por los aplausos ni quedarse con los éxitos obtenidos. Este compromiso con su arte y su público ha sido una de las cosas que la han mantenido relevante y respetada en el mundo cultural.

Ana Belén encontró en el cine un **vericueto** inesperado hacia el universo de las artes. Aunque había comenzado en la música, su experiencia en el cine infantil la llevó a descubrir nuevas partes de sí misma y de su talento. Fue en este recorrido donde desarrolló una capacidad única para tomar diferentes personajes y conectarse emocionalmente con el público. Este camino no solo fue un camino profesional, sino también una oportunidad para explorar sus habilidades como intérprete, algo que marcaría profundamente su carrera.

A los doce años, empezó a estudiar música en una academia conocida de Madrid. A los trece, participó en una película musical para niños que tuvo mucho éxito. El cine fue su **vericueto** hacia el teatro. Su éxito en el cine juvenil le dio confianza para intentar nuevos proyectos. Aunque era joven, Ana demostró gran profesionalismo y un talento natural. Su habilidad para conectar con el público desde muy joven prometía una carrera en las artes.

En 1971 conoció al cantante Víctor Manuel durante una gira de teatro en Galicia. Pronto actuaron juntos en una película. En 1972, Ana y Víctor se casaron en Gibraltar. Después de casarse, Ana volvió a cantar y lanzó su primer disco con canciones de Víctor. La **alquimia** de su música encantó a muchos.

Ana Belén ha trabajado mucho en cine, y con gran **entereza** ha sido también directora de teatro y actriz de televisión. Su trabajo es tan bueno que Francia le dio un premio importante (la Orden de las Artes) en 1986 y España le dio otro (la Medalla de Oro) en 2007. La **alquimia** que crea entre estas tres formas de actuar la hace muy especial.

Ahora, Ana no trabaja tanto y puede disfrutar más. Le encanta ir al gimnasio temprano y también disfruta ir al cine. Además, le gusta ver exposiciones de pintura que muestran **tornasol** de colores. Ana pasas mucho su tiempo con su familia y amigos, y después de tantos años de **desvelo** y esfuerzo, finalmente siente que puede usar su tiempo a lo que realmente ama. Su vida es más tranquila ahora. Cada día aprecia más los pequeños momentos que la vida le regala y sonríe al recordar el camino recorrido.

A lo largo de su carrera, Ana enfrentó muchos retos, pero siempre mostró gran **entereza**. Su carrera fue como una **lisonja** de logros. Para Ana, cada desafío ha sido una oportunidad para crecer. Considera que la fortaleza que mostró durante los momentos difíciles fue clave en su evolución como artista y persona. Su historia es una fuente de inspiración para todos aquellos que persiguen sus sueños a pesar de los obstáculos.

Durante su carrera, Ana Belén ha sido una **incógnita** fascinante para sus seguidores y críticos. Siempre buscó nuevas ideas para crear y exploró diferentes artes. Nunca se quedó en un solo estilo, y esa evolución constante la ha mantenido relevante con el tiempo. Su vida estuvo llena de **embeleso** por el arte. Para muchos, cada nuevo proyecto de Ana es una sorpresa, un misterio por descubrir que la ha hecho una de las personas más interesantes de la cultura española. Ella sigue siendo una **incógnita** en el mundo del arte. Ana Belén es una artista única, llena de pasión y compromiso. Su camino muestra que con esfuerzo y amor por el arte se puede lograr mucho."""

cleaned_text = re.sub(r'[^\w\s]', '', text) 
print(cleaned_text)
filtered_words = cleaned_text.lower().split()
words_that_are_too_rare = set()
for word in set(filtered_words):
    if get_word_frequency_percentiles([word], "Spanish")[word] > 20 and word.lower() not in rare_words and word.lower() not in ['didnt', 'isnt'] and not word.isdigit():
        print("-----------------------------")
        print(f"Encircled word '{word}' is treated as rare")
        base_word = find_base_word(word)
        base_word = re.sub(r'[^\w\s]', '', base_word)
        print(f"Base word for '{word}': {base_word}")
        if get_word_frequency_percentiles([base_word], "Spanish").get(base_word, 30) > 20:
            print("################")
            print("Still a rare word")
            print(f"Base word for '{word}': {base_word}")
            words_that_are_too_rare.add((word, f"Rank: {get_word_frequency_percentiles([base_word], TARGET_LANGUAGE).get(base_word, 30)}"))

        print("-----------------------------")

encircled_words = find_and_count_encircled_words(text)
print(encircled_words)
words_that_are_too_rare

Ana Belén una cantante directora y actriz famosa en España ha lanzado más de treinta y cinco discos actuado en treinta películas y veinte obras de teatro Ha dado conciertos por toda España y Latinoamérica dejando un rastro tornasol de emociones en cada presentación Ana ha defendido causas sociales a través de su arte interpretando canciones sobre paz derechos humanos y justicia social despertando un embeleso en su público que va más allá de la música Esto la ha convertido en una figura respetada tanto en el mundo artístico como en los movimientos de cambio

A lo largo de su carrera Ana Belén ha recibido muchos reconocimientos por su talento en las artes Sin embargo ella nunca ha considerado estos honores como una simple lisonja para Ana cada premio representa un reconocimiento al esfuerzo y al desvelo constante en su carrera Su enfoque ha sido siempre seguir aprendiendo y mejorando sin dejarse llevar por los aplausos ni quedarse con los éxitos obtenidos Este compromiso con su arte y su

KeyboardInterrupt: 

In [263]:
words_that_are_too_rare

{('guitarrista', 'Rank: 35.2'), ('latina', 'Rank: 32.5')}

In [206]:
find_base_word("adaptar")

'adaptar'

In [65]:
get_word_frequency_percentiles(["combin"], TARGET_LANGUAGE)

{'combinar': 37.9}

In [41]:
def repurpose_article(article):
    system_prompt = f"""You are language expert. Your task is to:

# Guidelines:
1. Rewrite the article using a very simple {TARGET_LANGUAGE}, every word in the text should be understandable by a B1-level language learner.

# Parameters:
- Article: The original article to simplify"""

    user_prompt = f"""Please follow these steps:
1. Rephrase this article using very simple {TARGET_LANGUAGE}, the entire article should be understandable by a beginner language learner, avoid any even slightly uncommon words. You can add paragraphs if needed.

# Parameters:
- Article: {article}"""

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": system_prompt.strip()},
            {"role": "user", "content": user_prompt.strip()}
        ]
    )

    return response.choices[0].message.content.strip()


text = """Sergio Martín es un joven y prometedor futbolista español, conocido por su habilidad en el campo, su dedicación, y sus rápidos reflejos. Con solo diecisiete años, ya ha comenzado a destacarse en las ligas juveniles, donde ha mostrado habilidades técnicas y tácticas notables. Sergio sueña con llegar a los niveles más altos del fútbol profesional y está decidido a hacer todo lo posible para lograrlo.

Nacido en 2007 en Zaragoza, Sergio creció en el seno de una familia futbolera. Es el mayor de dos hermanos, y sus padres, grandes seguidores del fútbol, siempre lo apoyaron en su pasión desde que era un niño. Desde pequeño, Sergio mostró interés en el deporte y pasó incontables horas jugando en los parques de su barrio con sus amigos y practicando con su padre, quien también había jugado en ligas menores en su juventud. Su madre, que veía su determinación y amor por el juego, fue quien lo inscribió en su primer club de fútbol a la temprana edad de seis años.

Con el paso del tiempo, Sergio fue perfeccionando sus habilidades y desarrollando una capacidad única para leer el juego y anticipar las jugadas. A los doce años, ya estaba jugando en un equipo juvenil en Zaragoza y comenzaba a recibir el interés de clubes mayores. Poco después, ingresó a una academia de fútbol muy conocida en su ciudad, donde recibió una formación más profesional y rigurosa. En esta academia, trabajó bajo la supervisión de entrenadores experimentados, quienes le enseñaron no solo la técnica, sino también la disciplina y el compromiso que exige el fútbol de alto rendimiento.

A los catorce años, Sergio fue seleccionado para jugar en el equipo juvenil de un club de segunda división en España, donde comenzó a competir a un nivel más alto y a viajar por distintas ciudades para enfrentarse a equipos de otras regiones. Esta experiencia le permitió conocer a jóvenes futbolistas de todo el país y aprender de sus estilos y habilidades, lo cual enriqueció su propio estilo de juego. Además, durante este periodo, Sergio se dio cuenta de la importancia de mantener una buena condición física y comenzó a ir al gimnasio, enfocándose en fortalecer su resistencia y velocidad.

El esfuerzo de Sergio en el campo no pasó desapercibido. Pronto llamó la atención de clubes de primera división, y a los dieciséis años, fue invitado a hacer una prueba en uno de estos equipos. Aunque no logró unirse al equipo principal, Sergio vio esta oportunidad como una valiosa experiencia y un recordatorio de que siempre hay margen para mejorar. Motivado por esta experiencia, regresó a su club con más determinación que nunca y continuó entrenando intensamente.

En 2024, Sergio fue seleccionado para participar en un torneo juvenil en Francia, representando a España junto con otros jóvenes talentos. Durante este torneo, Sergio destacó por su velocidad y habilidad para mover el balón, lo que le ganó el reconocimiento de varios entrenadores internacionales. Fue durante este evento que comenzó a soñar con la posibilidad de un día representar a su país en competiciones internacionales, como la Eurocopa o la Copa del Mundo.

A lo largo de su carrera juvenil, Sergio ha tenido la suerte de contar con el apoyo de su familia y amigos, quienes lo han acompañado en cada paso de su camino. Su padre siempre está presente en sus partidos, apoyándolo desde la grada y ofreciéndole consejos basados en su propia experiencia como futbolista. Su madre, quien ha sido su mayor admiradora, siempre le recuerda la importancia de disfrutar del juego y mantener los pies en la tierra.

Hoy en día, Sergio continúa entrenando y jugando con su equipo juvenil, pero también está trabajando en su formación académica, ya que sus padres insisten en la importancia de tener una base sólida fuera del fútbol. Aunque sueña con un futuro en el fútbol profesional, Sergio sabe que es fundamental tener otras opciones y desarrollar habilidades que le sirvan dentro y fuera del campo.

Cuando no está entrenando, Sergio disfruta de pasar su tiempo libre explorando otras aficiones, como la música y el cine. Le gusta ir al cine con sus amigos, asistir a conciertos locales, y también es un apasionado de la lectura, especialmente de biografías de otros futbolistas a quienes admira. Además, ha descubierto una nueva pasión por la cocina, una habilidad que le enseñó su abuela y que disfruta practicando en casa.

Con un talento natural, una gran ética de trabajo y un equipo de apoyo sólido, Sergio Martín está en el camino correcto para hacer realidad su sueño. Aunque aún le queda un largo camino por recorrer, su dedicación y compromiso con el fútbol le dan confianza de que, algún día, podrá llegar a formar parte de la élite del fútbol español."""

article_text_with_rare_words = repurpose_article(text)

print(article_text_with_rare_words)

Sergio Martín es un joven futbolista de España. Es muy bueno en el campo de fútbol. Tiene solo diecisiete años y ya juega muy bien en las ligas de jóvenes. Sergio quiere llegar a ser un jugador profesional muy importante.

Sergio nació en 2007 en Zaragoza. Sus padres y su hermano menor aman el fútbol. Desde pequeño, sus padres lo ayudaron mucho. Siempre le gustó jugar al fútbol. Pasaba muchas horas jugando en los parques con sus amigos y su papá. Su mamá lo inscribió en su primer club a los seis años.

Con el tiempo, Sergio mejoró sus habilidades. Él aprendió a entender el juego muy bien. A los doce años, jugaba en un equipo juvenil en Zaragoza. Los clubes más grandes empezaron a interesarse por él. Luego, entró en una academia de fútbol muy buena. Allí, aprendió mucho con entrenadores que tenían mucha experiencia.

A los catorce años, Sergio jugó en el equipo juvenil de un club de segunda división en España. Viajó a muchas ciudades y jugó contra otros equipos. Esto le ayudó a conocer 